# Labeled Property Graph Play

The notebook looks at working with the LPG from SysML v2 clients

In [1]:
import pymbe.api as pm
from pymbe.label import get_label
import networkx as NX
import matplotlib.pyplot as plt

from pymbe.interpretation.interp_playbooks import random_generator_playbook
from pymbe.interpretation.results import pprint_interpretation

from pymbe.query.metamodel_navigator import map_inputs_to_results

In [2]:
pm.UI.new(host_url="http://sysml2-sst.intercax.com")

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {()}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureMembership', 'FeatureValue', 'ParameterMembership', 'ResultExpressionMembership', 'ReturnParameterMembership')}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


DockPop(children=(UI(children=(SysML2ClientWidget(children=(Text(value='http://sysml2-sst.intercax.com', descr…

In [3]:
ui, *_ = _.children
client, tree, inspector, lpg, interpreter = ui.children
client.project_selector.value = client.project_selector.options["Kerbal (2021-04-17 18:03:01)"]
client._download_elements()

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureMembership', 'FeatureValue', 'Import', 'Membership', 'ParameterMembership', 'Redefinition', 'ResultExpressionMembership', 'ReturnParameterMembership', 'Superclassing')}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureMembership', 'FeatureTyping', 'FeatureValue', 'Import', 'Membership', 'ParameterMembership', 'Redefinition', 'ResultExpressionMembership', 'ReturnParameterMembership')}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")
c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureTyping', 'FeatureValue', 'Import', 'Membership', 'ParameterMembership', 'Redefinition', 'ResultExpressionMembership', 'ReturnParameterM

In [4]:
lpg.graph

In [5]:
shorten_pre_bake = {
    'RT-10 "Hammer" Solid Fuel Booster': "RT-10",
    'RT-5 "Flea" Solid Fuel Booster': "RT-5",
    'LV-T45 "Swivel" Liquid Fuel Engine': "LV-T45",
    'FL-T100 Fuel Tank': "FL-T100",
    'FL-T200 Fuel Tank': "FL-T200"
}

In [6]:
random_rez = random_generator_playbook(
    lpg,
    shorten_pre_bake
)

In [7]:
map_inputs_to_results(lpg)

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:265: UserWarning: These edge types are not in the graph: {('FeatureTyping', 'Import', 'Membership', 'Redefinition', 'Superclassing', 'FeatureMembership', 'FeatureValue', 'ParameterMembership', 'ResultExpressionMembership')}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


[('4197ab70-699b-4f69-9694-a5f39c2bb0b0',
  '263efbd5-75c4-4724-828c-e7f1e31a9e31',
  'ImpliedParameterFeedforward'),
 ('dbed26c7-1a89-4e04-ae50-d81ad4886ad6',
  '48b8fb0f-e769-4d09-999a-06bfcd0e4a12',
  'ImpliedParameterFeedforward'),
 ('74c492cc-1104-4ddd-b54c-5fd8a7cab158',
  '10605770-ea8a-4336-af28-6ab11ffe816a',
  'ImpliedParameterFeedforward'),
 ('ac79381e-74ee-4364-9d20-424a50f4834d',
  '28c81834-8712-4325-bdcd-f3d9487ad39b',
  'ImpliedParameterFeedforward'),
 ('bd31f91a-43a9-42f9-8e69-78c8dd16a03e',
  '615ed0f4-622b-499e-ade3-62ec6ede4517',
  'ImpliedParameterFeedforward'),
 ('57be4b8a-285d-41fc-a059-f8e1c3f68b58',
  '62c449d0-dc7a-4c43-b026-31fbdfb262d7',
  'ImpliedParameterFeedforward'),
 ('d146a0ab-de0c-4d8c-bd25-9913bde2f2e5',
  '3cfd0006-5b58-4891-a461-821d5190f9e3',
  'ImpliedParameterFeedforward'),
 ('ac16b279-fa4d-4ff4-8584-17e84ea401a2',
  'c2602a9d-5f1b-4296-a799-91dd7916f87f',
  'ImpliedParameterFeedforward')]

Want to look at all the \$results and other parameters

In [8]:
random_rez['ac79381e-74ee-4364-9d20-424a50f4834d']

[[LS#0,
  FM#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  FRE.Mass (p) => $result,
  FRE.Mass,
  LS#0.FM#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.FRE.Mass (p) => $result.FRE.Mass.$result#0 (unset)],
 [LS#1,
  FM#1,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  FRE.Mass (p) => $result,
  FRE.Mass,
  LS#1.FM#1.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.FRE.Mass (p) => $result.FRE.Mass.$result#1 (unset)],
 [LS#2,
  FM#2,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  FRE.Mass (p) => $result,
  FRE.Mass,
  LS#2.FM#2.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.FRE.Mass (p) => $result.FRE.Mass.$result#2 (unset)]]